# Realtime Video Object Detection
**This project is about realtime object detection in real time video files. Computer vision is a huge part of the data science/AI domain which is substantially advanced over the last couple of years.**


## Object Detection Task
The object detection task is based on following two prerequisite stages:
- Image classification: a supervised learning process to train an algorithm to recognize a set of target classes (objects to identify in images) using labeled example photos.
- Image classification with localization: the process of training a supervised algorithm to predict class as well as the bounding box around the object in the image. The term 'localization' refers to identifying the location of the object in the image. 

The object detection task refers to the situation where the model needs to detect multiple objects in the picture and localized them all by defining a bounding box around them. A prime example is an autonomous driving application, which needs to detect not just other cars, but also other pedestrians and motorcycles and even other objects. 

## YOLO algorithm
In recent years, deep learning algorithms are offering cutting-edge improved results for object detection. YOLO algorithm is one of the most popular Convolutional Neural Networks with a single end-to-end model that can perform object detection in real-time. YOLO stands for, You Only Look Once and is an algorithm developed by Joseph Redmon, et al. and first described in the 2015 paper titled “You Only Look Once: Unified, Real-Time Object Detection. The creation of the algorithm stemmed from the idea to place down a grid on the image and apply the image classification and localization algorithm to each of the grids.
Here the YOLOv3, a refined design which uses predefined anchor boxes to improve bounding box, is utilized for object detection in new images. Source code and pre-trained models of YOLOv3 is available in the official DarkNet GitHub repository.

### Tiny-YOLOv3
Tiny YOLOv3 is a simplified version of YOLOv3. It is a fast real-time detection algorithm developed for embedded devices. The model structure is simple and the accuracy is not as high as the full version. A higher miss detection rate is expected for small targets. This model was trained with the Coco data set and can detect up to 80 classes. 

#### Download the Pre-Trained Model
The first step is to download the pre-trained model weights using the DarkNet code base on the COCO dataset and place them into current working directory with the filename “yolov3-tiny.weights”.

- **YOLOv3-tiny Pre-trained Model Weights can be downloaded at** <a href="https://pjreddie.com/media/files/yolov3-tiny.weights" target="_blank">yolov3-tiny.weights(34 MB)</a>.


## OpenCV
OpenCV is an open source library which provides tools to perform image and video processing for the computer vision, machine learning, and image processing applications. It is particularly popular for real-time operations which is very important in today’s systems. Integration with various libraries, such as Numpuy and python resulted in great capablities of processing the OpenCV array structures for analysis and mathematical operations.

In [16]:
import cv2
from IPython.display import clear_output
import numpy as np
import time

OpenCV is one of the best computer vision libraries and it has functionalities for running deep learning inference. The OpenCV DNN module supports deep learning inference on images and videos. However, it does not support fine-tuning and training. 

OpenCV DNN module is highly optimized for Intel processors and can achieve high FPS when running inference on real-time videos for object detection and image segmentation applications. The high FPS with the DNN module are carried out using pre-trained models such as Tiny-YOLOv3.

In [17]:
# Load Yolo
net = cv2.dnn.readNet("yolov3-tiny.weights", "yolov3-tiny.cfg")

classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

Now, let’s take a look at video processing using an OpenCV and Python:
First of all, we are creating a cv2.VideoCapture object, cv2.VideoCapture is a class for video capturing from video files, image sequences, or cameras.

In [18]:
# Loading image
cap = cv2.VideoCapture("videos/traffic_video.mp4")

In [19]:
font = cv2.FONT_HERSHEY_PLAIN
starting_time = time.time()
frame_id = 0
imgarr = []

## Processing a video
Processing a video means, performing operations on the video frame by frame. Frames are nothing but just the particular instance of the video in a single point of time. Therefore, can be treated as an regular image.

The operations on frames started with cap.read() which returns a bool (True/False). If the frame is read correctly, it will be True. So you can check for the end of the video by checking this returned value.

Each frame is processed similar to an image and the object detection steps are similar to what was described in my previous project **Image Object Detector** (see the following github link)
https://github.com/majid-hosseini/Image-Object-Detector


The object detection process plot bounding boxed and the class names around each detected object and the processed frame is displayed using cv2.imshow method. We specify a window name as a first argument, and the frame we would like to display as a second. The time between consecutive frame in display window is defined by cv.waitKey(). An small value results in a fast video display and a large value produce a show motion video display.

## Saving a Video
We processed a video frame-by-frame and plotted bounding box and the class name around each detected object, now we want to save that video. For images, it is very simple: just use cv.imwrite(). Here, a little more work is required.

This time we create a VideoWriter object. We should specify the output file name (eg: output.avi). Then we should specify the FourCC code. Then number of frames per second (fps) and frame size should be passed. And the last one is the isColor flag. If it is True, the encoder expect color frame, otherwise it works with grayscale frame.

cap.release() and cv2.destroyAllWindows() are the methods to close video files or the capturing device, and destroy the window, which was created by the imshow method.

In [20]:
#cap= WebCamVideoStream(src=0).start()
(h, w) = (0, 0)
while True:
    _,frame= cap.read() # 
    frame_id+=1
    if frame is None:
        print("Loop finished...BREAKING")
        break
    clear_output()
    print(frame_id)
    Width = frame.shape[1]
    Height = frame.shape[0]
    scale = 0.00392    

    blob = cv2.dnn.blobFromImage(frame, scale, (416,416), (0,0,0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.4
    nms_threshold = 0.3

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                # Object detected
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)

                # Rectangle coordinates
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])


    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    
    for i in range(len(boxes)):
        if i in indices:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame,    (round(x),round(y)), (round(x+w),round(y+h)), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (round(x), round(y) + 30), font, 1.5, color, 2)
                

    (h, w) = frame.shape[:2]

    elapsed_time = time.time() - starting_time
    fps=frame_id/elapsed_time
    cv2.putText(frame,"FPS:"+str(round(round(fps,2)+8)),(50,50),font,2,(0,0,0),1)
    cv2.imshow("Image", frame)
    
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
        
    imgarr.append(frame)
cap.release()

388
Loop finished...BREAKING


In [21]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
print(h,w)
out = cv2.VideoWriter('output.mp4',fourcc, 30.0,(w,h))
for j in range (len(imgarr)):
    clear_output()
    out.write(imgarr[j])
    print('Frames Processed:',j,'/',frame_id)
out.release()
cv2.destroyAllWindows()

Frames Processed: 387 / 389
